#### Лабораторная работа № 1. Разработка нейросетевых функций. Операция Convolution 2D
#### БВТ2003 Суроп Максим Андреевич

In [29]:
import torch
import numpy as np
from typing import Union

In [30]:
class Conv2D:
    def __init__(self, in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple] = 1) -> None:
        # Инициализация параметров сверточного слоя
        self.in_channels, self.out_channels = in_channels, out_channels
        # Преобразование размера ядра и шага в кортеж, если они заданы в виде целых чисел
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = (stride, stride) if isinstance(stride, int) else stride

        # Инициализация весов (ядер свертки) и смещений
        self.weights = np.random.randn(out_channels, in_channels, *self.kernel_size)
        self.bias = np.zeros((out_channels,))

    def forward(self, input_tensor: np.ndarray) -> np.ndarray:
        # Извлечение размеров входного тензора
        batch_size, in_channels, input_height, input_width = input_tensor.shape
        kernel_size, stride = self.kernel_size, self.stride

        # Проверка корректности размера ядра относительно входного тензора
        if input_height < kernel_size[0] or input_width < kernel_size[1]:
            raise ValueError("Неверный размер ядра для входного тензора")

        # Извлечение размеров окна свертки и выходного тензора
        stride_h, stride_w = stride
        output_height = (input_height - kernel_size[0]) // stride_h + 1
        output_width = (input_width - kernel_size[1]) // stride_w + 1

        # Инициализация результата свертки
        result = np.zeros((batch_size, self.out_channels, output_height, output_width))

        # Выполнение операции свертки
        for i in range(0, input_height - kernel_size[0] + 1, stride_h):
            for j in range(0, input_width - kernel_size[1] + 1, stride_w):
                # Извлечение окна свертки из входного тензора
                window = input_tensor[:, :, i:i + kernel_size[0], j:j + kernel_size[1]]

                # Вычисление свертки и добавление смещения
                result[:, :, i // stride_h, j // stride_w] = np.sum(window * self.weights, axis=(1, 2, 3)) + self.bias

        return result


In [31]:
def run_test(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias, padding_mode):
    # Создание экземпляра Conv2D
    conv_layer = Conv2D(in_channels, out_channels, kernel_size, stride)

    # Создание тензора для теста
    input_tensor = np.random.randn(1, in_channels, 10, 10)

    # Выполнение свертки собственной реализацией
    result_conv2d = conv_layer.forward(input_tensor)

    # Выполнение свертки с использованием PyTorch и преобразование в numpy
    input_tensor_torch = torch.tensor(input_tensor)
    output_tensor_torch = torch.nn.functional.conv2d(input_tensor_torch,
                                                      torch.tensor(conv_layer.weights),
                                                      bias=torch.tensor(conv_layer.bias),
                                                      stride=stride,
                                                      padding=padding,
                                                      dilation=dilation,
                                                      groups=groups).numpy()

    # Обрезание результатов для сравнения
    min_height, min_width = map(min, (result_conv2d.shape[2:], output_tensor_torch.shape[2:]))
    result_conv2d, output_tensor_torch = result_conv2d[:, :, :min_height, :min_width], output_tensor_torch[:, :, :min_height, :min_width]

    # Проверка совпадения результатов
    are_results_equal = np.allclose(result_conv2d, output_tensor_torch)

    # Вывод информации о тесте
    print(f"Тест с параметрами:\n  - in_channels: {in_channels}\n  - out_channels: {out_channels}\n  - kernel_size: {kernel_size}\n  - stride: {stride}\n  - padding: {padding}\n  - dilation: {dilation}\n  - groups: {groups}\n  - bias: {bias}\n  - padding_mode: {padding_mode}\n\nРезультат свертки совпадает: {are_results_equal}\n\n{'-' * 75}\n")


In [32]:
def test_params() -> None:
    # параметры тестирования для первой группы
    run_test(in_channels=8, out_channels=20, kernel_size=3, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

    # параметры тестирования для второй группы
    run_test(in_channels=8, out_channels=4, kernel_size=2, stride=(2, 3), padding=0, dilation=1, groups=1, bias=True, padding_mode='replicate')

    # параметры тестирования для третьей группы
    run_test(in_channels=5, out_channels=32, kernel_size=(3, 2), stride=(4, 3), padding=1, dilation=2, groups=1, bias=True, padding_mode='zeros')


In [33]:
# параметры тестирования для дополнительной группы
input_tensor = np.random.randn(1, 3, 5, 5)
conv_layer = Conv2D(in_channels=3, out_channels=2, kernel_size=3, stride=(2, 2))
output_tensor = conv_layer.forward(input_tensor)

print(f"Результат свертки (Conv2D):\n {output_tensor}")

Результат свертки (Conv2D):
 [[[[ 1.35409044  1.76470831]
   [-6.78434554 -3.28487237]]

  [[ 4.92295796 -5.82149661]
   [ 1.03260716 -3.8928171 ]]]]


In [34]:
# преобразование результатов в формат torch.Tensor для сравнения с PyTorch
input_tensor_torch = torch.tensor(input_tensor)
output_tensor_torch = torch.nn.functional.conv2d(input_tensor_torch, 
                                                  torch.tensor(conv_layer.weights), 
                                                  bias=torch.tensor(conv_layer.bias), 
                                                  stride=(2, 2)).numpy()

print(f"Результат свертки (PyTorch):\n {output_tensor_torch}")

Результат свертки (PyTorch):
 [[[[ 1.35409044  1.76470831]
   [-6.78434554 -3.28487237]]

  [[ 4.92295796 -5.82149661]
   [ 1.03260716 -3.8928171 ]]]]


In [35]:
test_params()

Тест с параметрами:
  - in_channels: 8
  - out_channels: 20
  - kernel_size: 3
  - stride: 1
  - padding: 0
  - dilation: 1
  - groups: 1
  - bias: True
  - padding_mode: zeros

Результат свертки совпадает: True

---------------------------------------------------------------------------

Тест с параметрами:
  - in_channels: 8
  - out_channels: 4
  - kernel_size: 2
  - stride: (2, 3)
  - padding: 0
  - dilation: 1
  - groups: 1
  - bias: True
  - padding_mode: replicate

Результат свертки совпадает: True

---------------------------------------------------------------------------

Тест с параметрами:
  - in_channels: 5
  - out_channels: 32
  - kernel_size: (3, 2)
  - stride: (4, 3)
  - padding: 1
  - dilation: 2
  - groups: 1
  - bias: True
  - padding_mode: zeros

Результат свертки совпадает: False

---------------------------------------------------------------------------

